<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Nebenläufigkeit</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>


# Nebenläufigkeit

Definition von Leslie Lamport (in *Time, Clocks, and the Ordering of Events*, 1976):

<blockquote>
Zwei Ereignisse sind *nebenläufig*, wenn keines das andere kausal beeinflussen kann.
</blockquote>

D.h., nebenläufige Ereignisse können in beliebiger Reihenfolge ausgeführt werden.


## Wann ist Nebenläufigkeit sinnvoll?

- Zum Verringern von Latenz und Erhöhen von Durchsatz
- Zum Ausnutzen mehrerer Prozessoren/Rechenkerne
- Zur Durchführung von Hintergrundaktivitäten


Wie kann Nebenläufigkeit realisiert werden?

- Interleaving (Zeitscheiben)
- Asynchrone Verarbeitung (Sonderfall von Interleaving)
- Parallele Verarbeitung


Wie kann Nebenläufigkeit realisiert werden?

- Interleaving (Zeitscheiben): Coroutines, ...
- Asynchrone Verarbeitung: Event-Loop, Async, ...
- Parallele Verarbeitung: Threads, Processes, Futures, ...

Aber: In Python bewirken Threads in der Regel eher ein Interleaving als echte
parallele Verarbeitung!


## Threads

Threads werden durch die Klasse `threading.Thread` gekapselt:

- `target` Initarg bestimmt die Funktion, die ausgeführt wird
- `Thread.start()` startet den Thread
- `Thread.


### Hintergrundverarbeitung

In [1]:
def wait_and_print():
    from time import sleep
    print("Starting...")
    sleep(10)
    print("Stopping...")

In [2]:
from threading import Thread

my_thread = Thread(target=wait_and_print)

In [3]:
my_thread.start()

Starting...


In [4]:
print("Hello, from main Thread!")
print("My thread is alive:", my_thread.is_alive())

Hello, from main Thread!
My thread is alive: True


In [5]:
my_thread.join()
print("This should run only after my_thread is done.")
print("My thread is alive:", my_thread.is_alive())

Stopping...
This should run only after my_thread is done.
My thread is alive: False



### Verringern von Latenz und Erhöhen von Durchsatz

In [6]:
from time import sleep
from random import random
import timeit


def simulate_processing_time(delta_time=0.1):
    sleep(random() * delta_time + delta_time)

In [7]:
def process_request(data, results, delta_time=0.1):
    simulate_processing_time(delta_time)
    # Is this correct?
    results.append(f"->{data}")

In [8]:
def process_requests_sequentially(num_requests):
    results = []
    for i in range(num_requests):
        process_request(i, results)
    return results

In [9]:
process_requests_sequentially(5)

['->0', '->1', '->2', '->3', '->4']

In [10]:
timeit.timeit(lambda: process_requests_sequentially(5), globals=globals(), number=10)

8.317811999993864

In [11]:
timeit.timeit(lambda: process_requests_sequentially(10), globals=globals(), number=10)

15.901626099999703

In [12]:
from threading import Thread


def process_requests_concurrently(num_requests):
    results = []
    threads = []
    for i in range(num_requests):
        thread = Thread(target=lambda: process_request(i, results))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [13]:
process_requests_concurrently(5)

['->0', '->1', '->2', '->4', '->3']

In [14]:
timeit.timeit(lambda: process_requests_concurrently(5), globals=globals(), number=10)

1.9156025999982376

In [15]:
timeit.timeit(lambda: process_requests_concurrently(10), globals=globals(), number=10)

1.9869801999957417

In [16]:
timeit.timeit(lambda: process_requests_concurrently(100), globals=globals(), number=10)

2.186324099995545

In [17]:
class MyThread(Thread):
    # Note `run()`is overridden, not `start()`!
    def run(self) -> None:
        # noinspection PyUnresolvedReferences
        process_request(*self._args, **self._kwargs)

In [18]:
def process_requests_concurrently_2(num_requests):
    results = []
    threads = [MyThread(args=(i, results)) for i in range(num_requests)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [19]:
process_requests_concurrently_2(5)

['->3', '->2', '->1', '->4', '->0']

In [20]:
timeit.timeit(lambda: process_requests_concurrently_2(5), globals=globals(), number=10)

1.9329856999975163

In [21]:
timeit.timeit(lambda: process_requests_concurrently_2(10), globals=globals(), number=10)

1.9880924000026425

In [22]:
timeit.timeit(lambda: process_requests_concurrently_2(100), globals=globals(),
              number=10)

2.1880644999982906


### Mehrere Threads und das GIL

In [23]:
def perform_computation(data, results, num_iterations=1_000_000):
    result = 0
    for i in range(num_iterations):
        result += 1
    results.append(f"->{data}: {result}")

In [24]:
def perform_computations_sequentially(num_requests):
    results = []
    for i in range(num_requests):
        perform_computation(i, results)
    return results

In [25]:
perform_computations_sequentially(5)

['->0: 1000000',
 '->1: 1000000',
 '->2: 1000000',
 '->3: 1000000',
 '->4: 1000000']

In [26]:
timeit.timeit(lambda: perform_computations_sequentially(5), globals=globals(),
              number=10)

2.7833215999999084

In [27]:
timeit.timeit(lambda: perform_computations_sequentially(10), globals=globals(),
              number=10)

5.543484400004672

In [28]:
def perform_computations_concurrently(num_requests):
    results = []
    threads = []
    for i in range(num_requests):
        thread = Thread(target=lambda: perform_computation(i, results))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [29]:
perform_computations_concurrently(5)

['->0: 1000000',
 '->1: 1000000',
 '->2: 1000000',
 '->3: 1000000',
 '->4: 1000000']

In [30]:
timeit.timeit(lambda: perform_computations_concurrently(5), globals=globals(),
              number=10)

2.7456697000015993

In [31]:
timeit.timeit(lambda: perform_computations_concurrently(10), globals=globals(),
              number=10)

5.464458499998727


In Python wird immer nur *ein* Python Thread ausgeführt, alle anderen Threads
existieren zwar, warten aber "bis sie an der Reihe sind". Daher bringt Multithreading
nur dann Vorteile, wenn z.B. auf Ein/Ausgabe-Operationen gewartet wird, nicht wenn
mehrere Berechnungen beschleunigt werden sollen!

## Workshop

- Notebook `workshop_410_concurrency`
- Abschnitt "Parallele Requests"


### Synchronisieren von Threads

Die nebenläufige Programmierung führt zu Problemen, die es in sequentiellen Programmen
nicht gibt:

In [32]:
def add_ones():
    global _result
    for i in range(10_000):
        tmp = _result + 1
        # if random() > 0.99:
        #     simulate_processing_time(0)
        _result = tmp

In [33]:
from threading import Thread

_result = 0
_threads = [Thread(target=add_ones) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\n_result = {_result}")


_result = 1000000


In [34]:
def append_one():
    global _result_list
    for i in range(100_000):
        _result_list.append(1)

In [35]:
from threading import Thread

_result_list = []
_threads = [Thread(target=append_one) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\nLength of _result_list: {len(_result_list)}")


Length of _result_list: 10000000



#### Barrieren (Barriers)

Mit Barrieren (Barriers) kann eine fixe Anzahl an Threads synchronisiert werden:

In [36]:
from threading import Barrier, Thread

_barrier = Barrier(2, timeout=5)

In [37]:
def server1():
    print("Server is starting!")
    simulate_processing_time(1.0)
    print("Server started up!")
    _barrier.wait()
    print("Server is serving!")

In [38]:
def client1():
    print("Client is starting!")
    _barrier.wait()
    print("Client is accessing server!")

In [39]:
_c = Thread(target=client1)
_c.start()

Client is starting!


In [40]:
_s = Thread(target=server1)
_s.start()

Server is starting!


In [41]:
_c.join()
_s.join()

Server started up!
Server is serving!
Client is accessing server!


In [42]:
_s = Thread(target=server1)
_s.start()

Server is starting!


In [43]:
_c = Thread(target=client1)
_c.start()

Client is starting!


In [44]:
_s.join()
_c.join()

Server started up!
Server is serving!
Client is accessing server!



#### Locks

Locks sind ein low-level Synchronisierungsmechanismus, mit dem man erzwingen kann,
dass nur ein Thread eine Resource nutzt:

In [45]:
from threading import Lock, Thread

_result_lock = Lock()

In [46]:
def add_ones_locked():
    global _result
    for i in range(10_000):
        with _result_lock:
            tmp = _result + 1
            if random() > 0.99:
                simulate_processing_time(0)
            _result = tmp

In [47]:
_result = 0
_threads = [Thread(target=add_ones_locked) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\n_result = {_result}")


_result = 1000000


In [48]:
def server2():
    _barrier.wait()
    print("Server is serving")
    print("Server is still serving")
    print("Server is serving even more data")

In [49]:
def client2():
    _barrier.wait()
    print("Client is accessing server")
    print("Client is still accessing server")
    print("Client is taking really long to access the server")

In [50]:
def run_tasks(task1, task2):
    thread1 = Thread(target=task2)
    thread1.start()

    thread2 = Thread(target=task1)
    thread2.start()

    thread1.join()
    thread2.join()

In [51]:
run_tasks(server2, client2)

Server is servingClient is accessing server
Client is still accessing server
Client is taking really long to access the server

Server is still serving
Server is serving even more data


In [52]:
from threading import Lock

_print_lock = Lock()

In [53]:
def server3():
    _barrier.wait()
    try:
        _print_lock.acquire()
        simulate_processing_time()
        print("Server is serving")
        print("Server is still serving")
        print("Server is serving even more data")
    finally:
        _print_lock.release()

In [54]:
def client3():
    _barrier.wait()
    if _print_lock.acquire(blocking=False):
        print("Client is accessing server")
        print("Client is still accessing server")
        print("Client is taking really long to access the server")
        _print_lock.release()
    else:
        print("WARNING: Could not acquire lock!!!")

In [55]:
run_tasks(server3, client3)

Server is serving
Server is still serving
Server is serving even more data


In [56]:
run_tasks(client3, server3)

Client is accessing server
Client is still accessing server
Client is taking really long to access the server
Server is serving
Server is still serving
Server is serving even more data


In [57]:
def server4():
    _barrier.wait()
    with _print_lock:
        print("Server is serving")
        print("Server is still serving")
        print("Server is serving even more data")

In [58]:
def client4():
    _barrier.wait()
    with _print_lock:
        print("Client is accessing server")
        print("Client is still accessing server")
        print("Client is taking really long to access the server")

In [59]:
run_tasks(server4, client4)

Server is serving
Server is still serving
Server is serving even more data
Client is accessing server
Client is still accessing server
Client is taking really long to access the server


In [60]:
run_tasks(client4, server4)

Client is accessing server
Client is still accessing server
Client is taking really long to access the server
Server is serving
Server is still serving
Server is serving even more data



#### Condition Variables

Condition Variables sind ein Synchronisierungsmechanismus, der auf Locks basiert, aber
einen zusätzlichen Mechanismus zur Koordination von Threads bietet: `notify()` (und
`notify_all()`):

Typischerweise verwendet man Condition Variables, wenn sich mehrere Threads einen
gemeinsamen Zustand teilen und sowohl lesend als auch schreibend darauf zugreifen
müssen:

- Threads, die den Zustand lesen wollen, verwenden `wait()` und warten damit bis der
  gewünschte Zustand erreicht ist
- Threads, die den Zustand schreiben, verwenden `notify()` oder `notify_all()` um
  eventuell wartende Threads über die Änderung zu benachrichtigen

In [61]:
from threading import Condition, Thread

In [62]:
def consumer(consumer_id, cv, items):
    print(f"Consumer {consumer_id} started...", flush=True)
    with cv:
        print(f"Consumer {consumer_id} waiting...", flush=True)
        wait_succeeded = True
        while True:
            while not items and wait_succeeded:
                wait_succeeded = cv.wait(timeout=1.0)
            if not wait_succeeded:
                print(f"Consumer {consumer_id} timed out...", flush=True)
                break
            print(f"Consumer {consumer_id} starts consuming...", flush=True)
            item = items.pop()
            simulate_processing_time(0.1)
            print(f"Consumer {consumer_id} ends consuming item {item}...", flush=True)

In [63]:
def producer(producer_id, cv, num_items, items):
    from random import randint
    print(f"Producer {producer_id} started...", flush=True)
    for _ in range(num_items):
        with cv:
            item = randint(100, 999)
            print(f"Producer {producer_id} is producing item {item}", flush=True)
            items.append(item)
            cv.notify()
            simulate_processing_time(0.05)

In [64]:
def run_producer_consumer(num_items, num_producers=1, num_consumers=1):
    threads = []
    items = []
    cv = Condition()
    for i in range(num_consumers):
        threads.append(Thread(target=consumer, args=(i + 1, cv, items)))
    for i in range(num_producers):
        threads.append(Thread(target=producer, args=(i + 1, cv, num_items, items)))
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [65]:
run_producer_consumer(2)

Consumer 1 started...
Producer 1 started...
Consumer 1 waiting...
Producer 1 is producing item 453
Producer 1 is producing item 857
Consumer 1 starts consuming...
Consumer 1 ends consuming item 857...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 453...
Consumer 1 timed out...


In [66]:
run_producer_consumer(6, num_producers=1, num_consumers=3)

Consumer 1 started...
Consumer 2 started...
Consumer 3 started...
Consumer 1 waiting...
Producer 1 started...
Consumer 2 waiting...
Consumer 3 waiting...
Producer 1 is producing item 906
Consumer 1 starts consuming...
Consumer 1 ends consuming item 906...
Producer 1 is producing item 338
Producer 1 is producing item 714
Producer 1 is producing item 334
Producer 1 is producing item 232
Producer 1 is producing item 283
Consumer 1 starts consuming...
Consumer 1 ends consuming item 283...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 232...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 334...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 714...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 338...
Consumer 2 timed out...
Consumer 3 timed out...
Consumer 1 timed out...


In [67]:
run_producer_consumer(4, num_producers=3, num_consumers=4)

Consumer 1 started...
Consumer 2 started...
Consumer 3 started...
Consumer 1 waiting...
Consumer 4 started...
Producer 1 started...
Producer 2 started...
Producer 3 started...
Consumer 2 waiting...
Consumer 3 waiting...
Consumer 4 waiting...
Producer 2 is producing item 524
Producer 2 is producing item 714
Producer 2 is producing item 414
Producer 2 is producing item 139
Producer 1 is producing item 240
Producer 1 is producing item 794
Producer 3 is producing item 733
Producer 3 is producing item 447
Consumer 1 starts consuming...
Consumer 1 ends consuming item 447...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 733...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 794...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 240...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 139...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 414...
Consumer 1 starts consuming...
Consumer 1 ends consuming item 714...
Consumer 1 

In [68]:
from queue import Queue, Empty

In [69]:
def producer(producer_id, q, num_items):
    print(f"Producer {producer_id} started...")
    for i in range(num_items):
        print(f"Producer {producer_id} produced item {producer_id}/{i}...")
        q.put(f"Item {producer_id}/{i}")
        simulate_processing_time(0.1)

In [70]:
def consumer(consumer_id, q, timeout=1.0):
    print(f"Consumer {consumer_id} started...")
    try:
        while True:
            item = q.get(block=True, timeout=timeout)
            print(f"Consumer {consumer_id} starting processing of item {item}...")
            simulate_processing_time(0.2)
            print(f"Consumer {consumer_id} done processing item {item}...")
    except Empty:
        print(f"Consumer {consumer_id} timed out...")

In [71]:
from threading import Thread
def run_producer_consumer_queue(num_items, num_producers=1, num_consumers=1):
    processes = []
    q = Queue()
    for i in range(num_consumers):
        processes.append(Thread(target=consumer, args=(i + 1, q)))
    for i in range(num_producers):
        processes.append(Thread(target=producer, args=(i + 1, q, num_items)))
    for process in processes:
        process.start()
    for process in processes:
        process.join()

In [72]:
run_producer_consumer_queue(4)

Consumer 1 started...Producer 1 started...
Producer 1 produced item 1/0...

Consumer 1 starting processing of item Item 1/0...
Producer 1 produced item 1/1...
Consumer 1 done processing item Item 1/0...
Consumer 1 starting processing of item Item 1/1...
Producer 1 produced item 1/2...
Producer 1 produced item 1/3...
Consumer 1 done processing item Item 1/1...
Consumer 1 starting processing of item Item 1/2...
Consumer 1 done processing item Item 1/2...
Consumer 1 starting processing of item Item 1/3...
Consumer 1 done processing item Item 1/3...
Consumer 1 timed out...


In [73]:
run_producer_consumer_queue(6, num_producers=1, num_consumers=3)

Consumer 1 started...
Consumer 2 started...
Consumer 3 started...
Producer 1 started...
Producer 1 produced item 1/0...
Consumer 1 starting processing of item Item 1/0...
Producer 1 produced item 1/1...
Consumer 2 starting processing of item Item 1/1...
Producer 1 produced item 1/2...
Consumer 3 starting processing of item Item 1/2...
Consumer 1 done processing item Item 1/0...
Producer 1 produced item 1/3...
Consumer 1 starting processing of item Item 1/3...
Consumer 2 done processing item Item 1/1...
Producer 1 produced item 1/4...
Consumer 2 starting processing of item Item 1/4...
Consumer 3 done processing item Item 1/2...
Consumer 1 done processing item Item 1/3...
Producer 1 produced item 1/5...
Consumer 3 starting processing of item Item 1/5...
Consumer 2 done processing item Item 1/4...
Consumer 3 done processing item Item 1/5...
Consumer 1 timed out...
Consumer 2 timed out...
Consumer 3 timed out...


In [74]:
run_producer_consumer_queue(2, num_producers=4, num_consumers=3)

Consumer 1 started...
Consumer 2 started...
Consumer 3 started...
Producer 1 started...
Producer 1 produced item 1/0...
Consumer 1 starting processing of item Item 1/0...
Producer 2 started...
Producer 2 produced item 2/0...
Consumer 2 starting processing of item Item 2/0...
Producer 3 started...
Producer 3 produced item 3/0...
Consumer 3 starting processing of item Item 3/0...
Producer 4 started...
Producer 4 produced item 4/0...
Producer 4 produced item 4/1...
Producer 2 produced item 2/1...
Producer 3 produced item 3/1...
Producer 1 produced item 1/1...
Consumer 1 done processing item Item 1/0...
Consumer 1 starting processing of item Item 4/0...
Consumer 2 done processing item Item 2/0...
Consumer 2 starting processing of item Item 4/1...
Consumer 3 done processing item Item 3/0...
Consumer 3 starting processing of item Item 2/1...
Consumer 1 done processing item Item 4/0...
Consumer 1 starting processing of item Item 3/1...
Consumer 3 done processing item Item 2/1...
Consumer 3 st